In [1]:
import time
import pyproj
import requests
import statistics

geodesic = pyproj.Geod(ellps="WGS84")

def overpass_request(overpass_url: str, overpass_query: str):
    response = requests.get(overpass_url, params={"data": overpass_query}, timeout=100)
    response.raise_for_status()
    data = response.json()
    return data


def way_id_to_azimuth(way_id: int) -> tuple[float, float, float]:

    overpass_url = "http://overpass-api.de/api/interpreter"

    overpass_query = f"""
    [out:json];
    way({way_id});
    out geom;
    """

    for _ in range(5):
        try:
            data = overpass_request(overpass_url, overpass_query)
            break
        except:
            print("exception")
            time.sleep(5)



    assert len(data["elements"]) == 1
    assert len(data["elements"][0]["geometry"]) == 5

    lats = [e["lat"] for e in data["elements"][0]["geometry"][:-1]]
    lons = [e["lon"] for e in data["elements"][0]["geometry"][:-1]]

    edge_details = []
    for index, (lat, lon) in enumerate(zip(lats, lons)):

        eastToWest = lons[index + 1 if index + 1 != 4 else 0] - lon < 0

        fwd_az, back_az, distance = geodesic.inv(
            lon,
            lat,
            lons[index + 1 if index + 1 != 4 else 0],
            lats[index + 1 if index + 1 != 4 else 0],
        )
        edge_details.append(
            dict(
                fwd_az=fwd_az, back_az=back_az, distance=distance, eastToWest=eastToWest
            )
        )

    edge_details = sorted(edge_details, key=lambda each: each["distance"], reverse=True)

    azimuths = []
    for long_edge in edge_details[:2]:
        if long_edge["eastToWest"]:
            azimuths.append(180 + long_edge["fwd_az"])
        else:
            azimuths.append(long_edge["fwd_az"])
    try:
        assert abs(azimuths[0]-azimuths[1])<0.15
    except:
        print(azimuths)
        print(way_id)
        raise

    central_lat = statistics.mean(lats)
    central_lon = statistics.mean(lons)
    azimuth = statistics.mean(azimuths)

    return central_lat, central_lon, azimuth

In [2]:
from tqdm import tqdm
import time

way_ids = \
[
    1396232705,
    1394005696,
    1393542313,
    1393551052,
    1395826153,
    1288383635,
    1380438661,
    1380442967,
    686061931,
    1460223209,
    1375307993,
    1483895190,
    1453292719,
    1483895625,
    1483895787,
    1180021303,
    1466587133,
    1483896860,
    1277777447,
    1429411085
]

summary = {}
for way_id in tqdm(way_ids):
    src_lat, src_lon, az = way_id_to_azimuth(way_id)
    summary[way_id]=dict(az=az, src_lat=src_lat, src_lon=src_lon)
    time.sleep(13)

 10%|█         | 2/20 [00:35<05:09, 17.22s/it]

exception


 15%|█▌        | 3/20 [01:01<05:58, 21.09s/it]

exception


 55%|█████▌    | 11/20 [03:08<02:10, 14.55s/it]

...

 80%|████████  | 16/20 [04:34<01:07, 16.89s/it]

exception


 90%|█████████ | 18/20 [05:11<00:34, 17.22s/it]

exception


100%|██████████| 20/20 [06:00<00:00, 18.02s/it]


In [4]:
import shapely.ops
import pyproj.crs
import math

from shapely.geometry import Point as P

trans = pyproj.Transformer.from_crs(
    pyproj.crs.CRS("epsg:4326"), pyproj.crs.CRS("epsg:2154"), always_xy=True
).transform

for way_id, e in summary.items():

    e["az_geodetic_south"] = e["az"] - 90

    e["dst_lon"], e["dst_lat"], _ = geodesic.fwd(e["src_lon"], e["src_lat"], e["az"] + 90, 1)
    [e["src_x"]], [e["src_y"]] = shapely.ops.transform(trans, P(e["src_lon"], e["src_lat"])).xy
    [e["dst_x"]], [e["dst_y"]] = shapely.ops.transform(trans, P(e["dst_lon"], e["dst_lat"])).xy

    e["az_2154_south"] = 180 / math.pi * math.atan((e["dst_x"] - e["src_x"]) / (e["dst_y"] - e["src_y"]))

    e["convincing_south_choice"] = "Geodeteic" if abs(e["az_geodetic_south"]) < 10/60 and abs(e["az_geodetic_south"]/e["az_2154_south"])<1/20 else \
                                   "EPSG:2154" if abs(e["az_2154_south"]) < 10/60 and abs(e["az_2154_south"]/e["az_geodetic_south"])<1/20 else "Neither"


In [5]:
import pandas as pd
df = pd.DataFrame.from_dict(summary, orient='index')[["az_geodetic_south","az_2154_south","convincing_south_choice"]]
df = df.reset_index()
df = df.rename(columns={"index":"way_id"})

In [6]:
df.to_markdown("summary.md",index=False)